In [431]:
import pandas as pd
import numpy as np
import re

In [432]:
# import parser
import spacy 
from spacy.matcher import PhraseMatcher 
nlp = spacy.load('en_core_web_sm')
matcher = PhraseMatcher(nlp.vocab)

# terms would be each list in the dict

In [449]:
import json 
with open ('2020_july_lexiconV4_7.json') as f1: # import the latest lexicon
    dic1 = json.load(f1)

In [451]:

# import the dataset 

file1 = '2020_FB_Pres_Ads_500_Topic_Gold_v1_test.xlsx'
file2 = '2020_FB_Pres_Posts_500_Topic_GOLD_v1_test.xlsx'
file3 = '2020_TW_Pres_Posts_500_Topic_Gold_v1_test.xlsx'
dirs = 'C:\\Users\\wangtao\\Documents\\Illumination\\data_set(2020july)\\'

# df1 = pd.read_excel(dirs+file3)

df1 = pd.read_csv('FBtest_1500.csv') # here we import combined the test dataset


In [453]:
# The column that contains the text should be uniformly renamed as "text" 
df1.rename(columns={'ad_creative_body':'text'}, inplace=True)

In [455]:
# check the dataset 
if df1['text'].isnull().sum():
    print (df1['text'].isnull().sum())
    df1 = df1[df1['text'].notnull()]
    df1.reset_index(drop=True,inplace=True)

1

In [457]:
# df1.tail()  # check the dataset

,text,Gold Label
1494,Yo invertiría en infraestructura para que pers...,no topic
1495,"“During his Wisconsin out-reach, it became cle...",no topic
1496,Another endorsement coming out of Iowa today! ...,no topic
1497,Wow! Thanks everyone for supporting Help Meado...,no topic
1498,Join us at the Spurs vs. Lakers on November 25...,no topic


In [458]:
# function that filter the urls and symbols in the text 

def filter_text(x):
    url = 'http[s]?://\S+'
    x = re.sub(url,'',x)
    x = re.sub("[^\w\s]",' ',x) # filter symbols
    x = re.sub("\s+",' ',x)
    
    ls=[w.lower() for w in x.split()] 
    
    return ' '.join(ls)

In [460]:
df1['text'] = df1['text'].astype(str).apply(lambda x: filter_text(x))

In [463]:
# the function that find the lexicon words in the text
def find_words(x,lexicon):
    topics= lexicon.keys()  
    doc = nlp(x) # nlp() is spaCy 2.2 English language model 
    words= []
    for t in topics:
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms] 
        matcher.add("TerminologyList", None, *patterns) # spaCy2.2 phrase matcher
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    if words:
        words = list(set(words))
        return ','.join(words)
    else:
        return('no words')
        

In [464]:
# tagging the topic in each message
def find_topic(x,lexicon):
    topics= lexicon.keys()    
    if x=='no words':
        return ''    
    if x != 'no words': 
        words = x.split(',')
        labels = []        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                labels.append(t)                
                #l = sorted(labels)        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [465]:
df1['words'] = df1['text'].astype(str).apply(lambda x: find_words(x,dic1))

In [467]:
df1['m_label'] = df1['words'].apply(lambda x: find_topic(x,dic))

In [468]:
df1['m_label'] = df1['m_label'].apply(lambda x: 'no topic' if x=='' else x)